# 🤖 MetaUniverse LLM API 测试

测试多厂商 LLM Provider 是否可用：OpenAI, Anthropic, Gemini, 智谱, DeepSeek, Qwen, Kimi

## 1. 加载环境变量

In [ ]:
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv()

# 查看配置的 providers
providers = []
if os.getenv('OPENAI_API_KEY'):
    providers.append('openai')
if os.getenv('ANTHROPIC_API_KEY'):
    providers.append('anthropic')
if os.getenv('GEMINI_API_KEY'):
    providers.append('gemini')
if os.getenv('ZHIPU_API_KEY'):
    providers.append('zhipu')
if os.getenv('DEEPSEEK_API_KEY'):
    providers.append('deepseek')
if os.getenv('QWEN_API_KEY'):
    providers.append('qwen')
if os.getenv('KIMI_API_KEY'):
    providers.append('kimi')

print(f"✅ 已配置的 Providers: {providers}")
print(f"📊 默认 Provider: {os.getenv('LLM_DEFAULT_PROVIDER', '未设置')}")
print(f"💰 每日预算: ${os.getenv('LLM_DAILY_BUDGET', '10')}")

## 2. 测试 OpenAI (GPT)

In [ ]:
import requests

def test_openai():
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        print("⚠️ 未配置 OPENAI_API_KEY")
        return
    
    base_url = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
    model = os.getenv('OPENAI_MODEL', 'gpt-4o')
    
    try:
        response = requests.post(
            f"{base_url}/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            json={
                "model": model,
                "messages": [{"role": "user", "content": "Hello, this is a test. Reply with 'OpenAI OK'"}],
                "max_tokens": 20
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['choices'][0]['message']['content']
            usage = data['usage']
            cost = (usage['prompt_tokens'] + usage['completion_tokens']) / 1000 * 0.005
            print(f"✅ OpenAI OK")
            print(f"   Model: {data['model']}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {usage['total_tokens']}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ OpenAI Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ OpenAI Exception: {e}")

test_openai()

## 3. 测试 Anthropic (Claude)

In [ ]:
def test_anthropic():
    api_key = os.getenv('ANTHROPIC_API_KEY')
    if not api_key:
        print("⚠️ 未配置 ANTHROPIC_API_KEY")
        return
    
    base_url = os.getenv('ANTHROPIC_BASE_URL', 'https://api.anthropic.com/v1')
    model = os.getenv('ANTHROPIC_MODEL', 'claude-3-5-sonnet-20241022')
    
    try:
        response = requests.post(
            f"{base_url}/messages",
            headers={
                "x-api-key": api_key,
                "anthropic-version": "2023-06-01",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "max_tokens": 20,
                "messages": [{"role": "user", "content": "Reply with 'Anthropic OK'"}]
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['content'][0]['text']
            usage = data['usage']
            cost = (usage['input_tokens'] + usage['output_tokens']) / 1000 * 0.003
            print(f"✅ Anthropic OK")
            print(f"   Model: {data['model']}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {usage['input_tokens'] + usage['output_tokens']}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ Anthropic Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ Anthropic Exception: {e}")

test_anthropic()

## 4. 测试 Google Gemini

In [ ]:
def test_gemini():
    api_key = os.getenv('GEMINI_API_KEY')
    if not api_key:
        print("⚠️ 未配置 GEMINI_API_KEY")
        return
    
    base_url = os.getenv('GEMINI_BASE_URL', 'https://generativelanguage.googleapis.com/v1beta')
    model = os.getenv('GEMINI_MODEL', 'gemini-1.5-pro')
    
    try:
        response = requests.post(
            f"{base_url}/models/{model}:generateContent?key={api_key}",
            headers={"Content-Type": "application/json"},
            json={
                "contents": [{"parts": [{"text": "Reply with 'Gemini OK'"}]}],
                "generationConfig": {"maxOutputTokens": 20}
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['candidates'][0]['content']['parts'][0]['text']
            usage = data.get('usageMetadata', {})
            total_tokens = usage.get('promptTokenCount', 0) + usage.get('candidatesTokenCount', 0)
            cost = total_tokens / 1000 * 0.0035
            print(f"✅ Gemini OK")
            print(f"   Model: {model}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {total_tokens}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ Gemini Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ Gemini Exception: {e}")

test_gemini()

## 5. 测试智谱清言 (Zhipu/GLM)

In [ ]:
def test_zhipu():
    api_key = os.getenv('ZHIPU_API_KEY')
    if not api_key:
        print("⚠️ 未配置 ZHIPU_API_KEY")
        return
    
    base_url = os.getenv('ZHIPU_BASE_URL', 'https://open.bigmodel.cn/api/paas/v4')
    model = os.getenv('ZHIPU_MODEL', 'glm-4')
    
    try:
        response = requests.post(
            f"{base_url}/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": [{"role": "user", "content": "回复'智谱OK'"}],
                "max_tokens": 20
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['choices'][0]['message']['content']
            usage = data['usage']
            cost = usage['total_tokens'] / 1000 * 0.0014
            print(f"✅ 智谱 OK")
            print(f"   Model: {data['model']}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {usage['total_tokens']}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ 智谱 Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ 智谱 Exception: {e}")

test_zhipu()

## 6. 测试 DeepSeek

In [ ]:
def test_deepseek():
    api_key = os.getenv('DEEPSEEK_API_KEY')
    if not api_key:
        print("⚠️ 未配置 DEEPSEEK_API_KEY")
        return
    
    base_url = os.getenv('DEEPSEEK_BASE_URL', 'https://api.deepseek.com/v1')
    model = os.getenv('DEEPSEEK_MODEL', 'deepseek-chat')
    
    try:
        response = requests.post(
            f"{base_url}/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": [{"role": "user", "content": "回复'DeepSeek OK'"}],
                "max_tokens": 20
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['choices'][0]['message']['content']
            usage = data['usage']
            cost = usage['total_tokens'] / 1000 * 0.00027
            print(f"✅ DeepSeek OK")
            print(f"   Model: {data['model']}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {usage['total_tokens']}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ DeepSeek Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ DeepSeek Exception: {e}")

test_deepseek()

## 7. 测试阿里云 Qwen

In [ ]:
def test_qwen():
    api_key = os.getenv('QWEN_API_KEY')
    if not api_key:
        print("⚠️ 未配置 QWEN_API_KEY")
        return
    
    base_url = os.getenv('QWEN_BASE_URL', 'https://dashscope.aliyuncs.com/api/v1')
    model = os.getenv('QWEN_MODEL', 'qwen-plus')
    
    try:
        response = requests.post(
            f"{base_url}/services/aigc/text-generation/generation",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "input": {
                    "messages": [{"role": "user", "content": "回复'Qwen OK'"}]
                },
                "parameters": {
                    "max_tokens": 20,
                    "result_format": "message"
                }
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['output']['choices'][0]['message']['content']
            usage = data['usage']
            total_tokens = usage['input_tokens'] + usage['output_tokens']
            cost = total_tokens / 1000 * 0.0008
            print(f"✅ Qwen OK")
            print(f"   Model: {model}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {total_tokens}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ Qwen Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ Qwen Exception: {e}")

test_qwen()

## 8. 测试 Kimi (Moonshot)

In [ ]:
def test_kimi():
    api_key = os.getenv('KIMI_API_KEY')
    if not api_key:
        print("⚠️ 未配置 KIMI_API_KEY")
        return
    
    base_url = os.getenv('KIMI_BASE_URL', 'https://api.moonshot.cn/v1')
    model = os.getenv('KIMI_MODEL', 'kimi-latest')
    
    try:
        response = requests.post(
            f"{base_url}/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json={
                "model": model,
                "messages": [{"role": "user", "content": "回复'Kimi OK'"}],
                "max_tokens": 20
            },
            timeout=30
        )
        
        if response.status_code == 200:
            data = response.json()
            content = data['choices'][0]['message']['content']
            usage = data['usage']
            cost = usage['total_tokens'] / 1000 * 0.003
            print(f"✅ Kimi OK")
            print(f"   Model: {data['model']}")
            print(f"   Response: {content[:50]}...")
            print(f"   Tokens: {usage['total_tokens']}")
            print(f"   Cost: ${cost:.4f}")
        else:
            print(f"❌ Kimi Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"❌ Kimi Exception: {e}")

test_kimi()

## 9. 批量测试所有 API

In [ ]:
print("\n" + "="*50)
print("🚀 批量测试所有已配置的 LLM API")
print("="*50 + "\n")

test_openai()
print()
test_anthropic()
print()
test_gemini()
print()
test_zhipu()
print()
test_deepseek()
print()
test_qwen()
print()
test_kimi()

print("\n" + "="*50)
print("✨ 测试完成！")
print("="*50)

## 10. 测试文章生成功能

使用实际的 Agent 技能生成一篇测试文章

In [ ]:
import json

def test_write_article():
    """测试 WriteArticle 技能"""
    default_provider = os.getenv('LLM_DEFAULT_PROVIDER', 'openai')
    
    print(f"📝 测试文章生成（使用 {default_provider}）...")
    
    # 构建测试请求
    topic = "什么是强化学习（简要介绍）"
    
    # 使用默认 provider 直接调用
    llm = get_llm_manager()
    
    messages = [
        {"role": "system", "content": "你是一个专业的技术博客作者。为给定的主题生成简短的测试文章（300字以内）。"},
        {"role": "user", "content": f"请写一篇关于 '{topic}' 的简短介绍文章："}
    ]
    
    try:
        response = llm.chat({"messages": messages})
        print(f"✅ 文章生成成功！")
        print(f"   Tokens: {response['usage']['total_tokens']}")
        print(f"   Cost: ${response['cost']:.4f}")
        print(f"\n📄 文章内容预览：")
        print(f"{response['content'][:300]}...")
    except Exception as e:
        print(f"❌ 文章生成失败: {e}")

def get_llm_manager():
    """获取 LLM Manager 实例"""
    # 这里简化处理，实际应该导入 TypeScript 模块
    # 在实际使用时，可以通过 Node.js 调用或浏览器控制台
    class MockLLMManager:
        def chat(self, request):
            # 调用实际的 API
            provider = os.getenv('LLM_DEFAULT_PROVIDER', 'openai')
            
            # 根据 provider 调用对应测试函数
            if provider == 'openai':
                api_key = os.getenv('OPENAI_API_KEY')
                base_url = os.getenv('OPENAI_BASE_URL', 'https://api.openai.com/v1')
                model = os.getenv('OPENAI_MODEL', 'gpt-4o')
                
                response = requests.post(
                    f"{base_url}/chat/completions",
                    headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
                    json={
                        "model": model,
                        "messages": request["messages"],
                        "max_tokens": 500
                    },
                    timeout=60
                )
                
                data = response.json()
                usage = data['usage']
                cost = (usage['prompt_tokens'] + usage['completion_tokens']) / 1000 * 0.005
                
                return {
                    'content': data['choices'][0]['message']['content'],
                    'usage': usage,
                    'cost': cost
                }
            else:
                raise NotImplementedError(f"Provider {provider} not implemented in test")
    
    return MockLLMManager()

# 运行测试
if os.getenv('OPENAI_API_KEY') or os.getenv('LLM_DEFAULT_PROVIDER'):
    test_write_article()
else:
    print("⚠️ 未配置 API Key，跳过文章生成测试")

---

## 📋 总结

运行以上测试后，你可以：

1. **确认 API 可用性** - 查看哪些 provider 可以正常调用
2. **估算成本** - 了解不同模型的 Token 价格和实际花费
3. **测试响应质量** - 检查各模型的回复质量
4. **验证配置** - 确保 .env 文件配置正确

如果某些测试失败，请检查：
- API Key 是否正确
- 账户是否有足够余额
- 网络连接是否正常
- Base URL 是否需要代理